# Importing the libraries

In [115]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.seasonal import STL, seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Getting the data

In [116]:
dataset = pd.read_excel('indicadoressegurancapublicauf.xlsx')
dataset.head(5)

,UF,Tipo Crime,Ano,Mês,Ocorrências
0,Acre,Estupro,2022,janeiro,31
1,Acre,Furto de veículo,2022,janeiro,50
2,Acre,Homicídio doloso,2022,janeiro,10
3,Acre,Lesão corporal seguida de morte,2022,janeiro,1
4,Acre,Roubo a instituição financeira,2022,janeiro,0


In [117]:
dataset.rename(columns={
  'UF': 'State', 
  'Tipo Crime': 'Crime',
  'Ano': 'Year',
  'Mês': 'Month',
  'Ocorrências': 'Occurrences'
  }, inplace=True)

dataset.columns

Index(['State', 'Crime', 'Year', 'Month', 'Occurrences'], dtype='object')

# Data Preparation

In [118]:
STATE = 'Rio Grande do Sul'

In [119]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23020 entries, 0 to 23019
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   State        23020 non-null  object
 1   Crime        23020 non-null  object
 2   Year         23020 non-null  int64 
 3   Month        23020 non-null  object
 4   Occurrences  23020 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 899.3+ KB


In [120]:
dataset.shape

(23020, 5)

In [121]:
dataset.columns

Index(['State', 'Crime', 'Year', 'Month', 'Occurrences'], dtype='object')

In [122]:
dataset.isnull().sum()

State          0
Crime          0
Year           0
Month          0
Occurrences    0
dtype: int64

In [123]:
dataset.isnull().sum()

State          0
Crime          0
Year           0
Month          0
Occurrences    0
dtype: int64

In [124]:
dataset[dataset['State'] == STATE]

,State,Crime,Year,Month,Occurrences
2081,Rio Grande do Sul,Estupro,2022,janeiro,408
2082,Rio Grande do Sul,Furto de veículo,2022,janeiro,708
2083,Rio Grande do Sul,Homicídio doloso,2022,janeiro,141
2084,Rio Grande do Sul,Lesão corporal seguida de morte,2022,janeiro,3
2085,Rio Grande do Sul,Roubo a instituição financeira,2022,janeiro,0
...,...,...,...,...,...
22379,Rio Grande do Sul,Roubo a instituição financeira,2015,dezembro,8
22380,Rio Grande do Sul,Roubo de carga,2015,dezembro,28
22381,Rio Grande do Sul,Roubo de veículo,2015,dezembro,1520
22382,Rio Grande do Sul,Roubo seguido de morte (latrocínio),2015,dezembro,11


In [125]:
state_dataset = dataset[dataset['State'] == STATE]
state_dataset.head(5)

,State,Crime,Year,Month,Occurrences
2081,Rio Grande do Sul,Estupro,2022,janeiro,408
2082,Rio Grande do Sul,Furto de veículo,2022,janeiro,708
2083,Rio Grande do Sul,Homicídio doloso,2022,janeiro,141
2084,Rio Grande do Sul,Lesão corporal seguida de morte,2022,janeiro,3
2085,Rio Grande do Sul,Roubo a instituição financeira,2022,janeiro,0


In [126]:
state_dataset.loc[:,'State'] = 'RS'
state_dataset

c:\Users\thier\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,State,Crime,Year,Month,Occurrences
2081,RS,Estupro,2022,janeiro,408
2082,RS,Furto de veículo,2022,janeiro,708
2083,RS,Homicídio doloso,2022,janeiro,141
2084,RS,Lesão corporal seguida de morte,2022,janeiro,3
2085,RS,Roubo a instituição financeira,2022,janeiro,0
...,...,...,...,...,...
22379,RS,Roubo a instituição financeira,2015,dezembro,8
22380,RS,Roubo de carga,2015,dezembro,28
22381,RS,Roubo de veículo,2015,dezembro,1520
22382,RS,Roubo seguido de morte (latrocínio),2015,dezembro,11


### Converting month from str to int

In [127]:
state_dataset['Month'].unique()

array(['janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho',
       'agosto', 'setembro', 'outubro', 'novembro', 'dezembro'],
      dtype=object)

In [128]:
month_dic = { 
 'janeiro': 1,
 'fevereiro': 2,
 'março': 3,
 'abril': 4,
 'maio': 5,
 'junho': 6,
 'julho': 7,
 'agosto': 8,
 'setembro': 9,
 'outubro': 10,
 'novembro': 11,
 'dezembro': 12
}

In [129]:
state_dataset['Month'] = state_dataset['Month'].map(month_dic)
state_dataset['Month']

C:/Users/thier/AppData/Local/Temp/xpython_13416/1514361170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2081      1
2082      1
2083      1
2084      1
2085      1
         ..
22379    12
22380    12
22381    12
22382    12
22383    12
Name: Month, Length: 864, dtype: int64

In [130]:
sorted_dataset = state_dataset.sort_values(by=['Year', 'Month'])
sorted_dataset.tail(12)

,State,Crime,Year,Month,Occurrences
2177,RS,Roubo de veículo,2022,11,381
2178,RS,Roubo seguido de morte (latrocínio),2022,11,1
2179,RS,Tentativa de homicídio,2022,11,171
2180,RS,Estupro,2022,12,444
2181,RS,Furto de veículo,2022,12,781
2182,RS,Homicídio doloso,2022,12,149
2183,RS,Lesão corporal seguida de morte,2022,12,3
2184,RS,Roubo a instituição financeira,2022,12,0
2185,RS,Roubo de carga,2022,12,9
2186,RS,Roubo de veículo,2022,12,349


### Compressing the name of the crimes

In [131]:
print(len(sorted_dataset['Crime'].unique()))
state_dataset['Crime'].unique()

9


array(['Estupro', 'Furto de veículo', 'Homicídio doloso',
       'Lesão corporal seguida de morte',
       'Roubo a instituição financeira', 'Roubo de carga',
       'Roubo de veículo', 'Roubo seguido de morte (latrocínio)',
       'Tentativa de homicídio'], dtype=object)

In [132]:
crime_dic = {
  'Estupro': 'Rape',
  'Furto de veículo': 'Stolen vehicle',
  'Homicídio doloso': 'Intentional homicide',
  'Lesão corporal seguida de morte': 'Bodily injury death',
  'Roubo a instituição financeira': 'Bank robbery',
  'Roubo de carga': 'Cargo Theft',
  'Roubo de veículo': 'Vehicle Theft',
  'Roubo seguido de morte (latrocínio)': 'Robbery Murder',
  'Tentativa de homicídio': 'Attempted Murder'
}

In [133]:
sorted_dataset['Crime'] = sorted_dataset['Crime'].map(crime_dic)
sorted_dataset['Crime']

22276                    Rape
22277          Stolen vehicle
22278    Intentional homicide
22279     Bodily injury death
22280            Bank robbery
                 ...         
2184             Bank robbery
2185              Cargo Theft
2186            Vehicle Theft
2187           Robbery Murder
2188         Attempted Murder
Name: Crime, Length: 864, dtype: object

In [135]:
sorted_dataset['Crime'].unique()

array(['Rape', 'Stolen vehicle', 'Intentional homicide',
       'Bodily injury death', 'Bank robbery', 'Cargo Theft',
       'Vehicle Theft', 'Robbery Murder', 'Attempted Murder'],
      dtype=object)

In [136]:
sorted_dataset = pd.get_dummies(sorted_dataset, columns=['Crime'])
sorted_dataset

In [137]:
sorted_dataset.columns

Index(['State', 'Year', 'Month', 'Occurrences', 'Crime_Attempted Murder',
       'Crime_Bank robbery', 'Crime_Bodily injury death', 'Crime_Cargo Theft',
       'Crime_Intentional homicide', 'Crime_Rape', 'Crime_Robbery Murder',
       'Crime_Stolen vehicle', 'Crime_Vehicle Theft'],
      dtype='object')

In [138]:
sorted_dataset

,State,Year,Month,Occurrences,Crime_Attempted Murder,Crime_Bank robbery,Crime_Bodily injury death,Crime_Cargo Theft,Crime_Intentional homicide,Crime_Rape,Crime_Robbery Murder,Crime_Stolen vehicle,Crime_Vehicle Theft
22276,RS,2015,1,502,0,0,0,0,0,1,0,0,0
22277,RS,2015,1,1644,0,0,0,0,0,0,0,1,0
22278,RS,2015,1,230,0,0,0,0,1,0,0,0,0
22279,RS,2015,1,2,0,0,1,0,0,0,0,0,0
22280,RS,2015,1,6,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2184,RS,2022,12,0,0,1,0,0,0,0,0,0,0
2185,RS,2022,12,9,0,0,0,1,0,0,0,0,0
2186,RS,2022,12,349,0,0,0,0,0,0,0,0,1
2187,RS,2022,12,4,0,0,0,0,0,0,1,0,0


* Criar date
